### Preliminary setup

In [ ]:
%pip install pyarrow

In [29]:
import pandas as pd             # For data transformation
import glob                     # For loading all Parquet files

from functools import reduce

### Reading Parquet files

In [25]:
bpi_parquet_files = glob.glob('base-datasets/BPI/*.parquet')
bpi_dfs = [pd.read_parquet(data) for data in bpi_parquet_files]

In [ ]:
for x in bpi_dfs:
    print(x.head())

In [28]:
print([x.shape for x in bpi_dfs])

[(143165, 14), (92204, 18), (35718, 10), (522409, 12), (100000, 3)]


Since the dimensions of each file are different, we merge entries with matching CUSTOMER_ID values to a central dataset.

In [ ]:
# bpi_merged = reduce(lambda left, right: pd.merge(left, right, on='CUSTOMER_ID', how='inner'), bpi_dfs)
# bpi_merged

In [39]:
# bpi_merged = bpi_merged.drop('CURRENT_MONTH_y', axis=1)

In [40]:
# bpi_merged = bpi_merged.drop_duplicates()

In [41]:
# bpi_merged.shape

(323532, 52)

In [50]:
# bpi_merged.to_csv()
bpi_dfs[0].to_csv('ccconso.csv')
bpi_dfs[1].to_csv('cdna.csv')
bpi_dfs[2].to_csv('loan.csv')
bpi_dfs[3].to_csv('products.csv')
bpi_dfs[4].to_csv('segments.csv')

In [44]:
# bpi_merged.shape

(323532, 52)